In [1]:
cd ~/PYTHON/ECOGpy/

/home/knight/matar/PYTHON/ECOGpy


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import os
from scipy import stats
from utils.loadmat import loadmat
from utils.fdr_correct import fdr2
import scipy.io as spio
import numpy as np

In [4]:
base = '/home/knight/matar/MATLAB/DATA/Avgusta/'
savedir = os.path.join(base, 'PCA', 'bins', 'bycluster')
savedir

'/home/knight/matar/MATLAB/DATA/Avgusta/PCA/bins/bycluster'

In [5]:
filename = os.path.join(base, 'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df = pd.read_csv(filename)

In [170]:
subj, task = 'GP15', 'EmoGen'

filename = os.path.join(base, 'Subjs', subj, task, 'HG_elecMTX_zscore.mat')
data_dict = loadmat(filename)
data = data_dict['data_zscore']
active_elecs = data_dict['active_elecs']
RTs = data_dict['RTs']

for c in df[(df.subj == subj) & (df.task == task)].cluster.unique():
    print c
    #get elecs in cluster
    elecs = df[(df.subj == subj) & (df.task == task)].groupby('cluster').get_group(c).elec.values
    eidx = np.in1d(active_elecs, elecs)
   
    #get cluster trials, sorted by rt
    cdata = np.vstack(data[eidx,:,:])
    RTs_tile = np.tile(RTs, sum(eidx))
    ridx = np.argsort(RTs_tile)
    cdata = cdata[ridx,:]

    #take average across elecs
    cdata = pd.DataFrame(cdata, index = np.repeat(range(data.shape[1]), sum(eidx)))
    cdata = cdata.groupby(cdata.index).mean()

    filename = os.path.join(savedir, 'trace_data', 'trace_data_c%i.mat' %(c))
    spio.savemat(filename, {'cdata': np.array(cdata), 'RTs': RTs[RTs.argsort()]})


4
5
6
3
1


In [169]:
filename

'/home/knight/matar/MATLAB/DATA/Avgusta/PCA/bins/bycluster/trace_data/trace_data_c1.mat'